# Problem
"By replacing the 1st digit of the 2-digit number *3, it turns out that six of the nine possible values: 13, 23, 43, 53, 73, and 83, are all prime.

By replacing the 3rd and 4th digits of 56**3 with the same digit, this 5-digit number is the first example having seven primes among the ten generated numbers, yielding the family: 56003, 56113, 56333, 56443, 56663, 56773, and 56993. Consequently 56003, being the first member of this family, is the smallest prime with this property.

Find the smallest prime which, by replacing part of the number (not necessarily adjacent digits) with the same digit, is part of an eight prime value family."

# Clarifications
I must find the lowest number such that, by simultaneously replacing some collection of like-digits with another digit, a total of 8 primes are generated.     
- we can't change the 1st digit to 0   
- if there are three like-digits, you can replace just two of them.

Strategy
Check whether each number has the property, starting at 3. Reduce time needed to check whether a new number is prime by adding primes to a list as they are found. A new number is prime just if it isn't divisible by any of the primes found so far.

In [51]:
#This function checks whether new numbers are prime, as we move up through 
#the integers, and collects any primes it finds into primes_list
#It allows us to efficiently check if a new number is prime.

def check_if_prime(num, primes_list, add = True):
    '''Returns True iff num is prime. 
    If add = True and num is prime, num is appended to primes_list.
    Only works if 
    -numbers are checked in ascending order
    -add=True throughout
    -all primes lower than than the first number checked are in primes at the start
    This is computationally more efficient than checking each number from scratch'''
    for n in primes_list:
        if num%n == 0:
            return False
    if add:
        primes_list.append(num)
    return True

In [77]:
primes_list = [2]
for m in range(3, 101):
    prime = True
    for n in primes_list:
        if m%n == 0:
            prime = False
    if prime:
        primes_list.append(m)
print primes_list

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [52]:
#This helps us search for families of primes by changing one digit of a number.
def change_digit(num, d, i):
    '''Change the dth digit of num to i. Return the new number.
    >>> change_digit(123, 1, 5)
    153'''
    old_str = str(num)
    new_str = ''
    for digit in range(d):
        new_str += old_str[digit]
    new_str += str(i)
    for digit in range(d+1, len(str(num))):
        new_str += old_str[digit]
    return int(new_str)
    

In [53]:
#I need a way of identifying when a digit is repeated in a number, and then changing the repeated digits simultaneously.
#This function does the second job. 
#This generalises change_digit for the case when I need to change many digits simultaneously.
def change_digits(num, indice_list, i):
    '''Change num such that its digits in indice_list turn to i. Return the new number.
    >>> change_digits(123456,[0,2] , 5)
    525456
    >>> change_digits(10000005,[1,3] , 9)
    19090005'''
    old_str = str(num)
    new_str = ''
    for d in range(len(str(num))): #go through num digit by digit
        if d in indice_list:
            new_str += str(i)     
        else:
            new_str += old_str[d]
    return int(new_str)

# What follows is my attempt to make a formula that takes a number and returns  the indices of each group of repeating digits.

In [55]:
#practice: this function returns the indices of PAIRS of repeating digits
# (I need a function that returns PAIRS or THREES or... of repeating digits) 
def find_digits_pairs(num):
    '''Returns a list containing the indices of each pair of identical digits in num.
    >>>find_digits_pairs(40404)
    [[0,2],[1,3],[0,4],[2,4]]'''
    list_ = []
    num_str = str(num)
    num_length = len(num_str)
    for d in range(num_length):
        for e in range(d+1, num_length):
            if num_str[d] == num_str[e]:
                list_.append([d, e])
    return list_

In [56]:
find_digits_pairs(40404)

[[0, 2], [0, 4], [1, 3], [2, 4]]

In [59]:
#practice: this function returns the indices of THREE of repeating digits
# (I need a function that returns groups of repeating digits of ANY size - not just 3) 
def find_digits_threes(num):
    '''Returns a list containing the indices of each group of three identical digits in num.
    >>>find_like_digits(404040, 3)
    [[0,2,4],[1,3,5]] '''
    list_ = []
    num_str = str(num)
    num_length = len(num_str)
    for d in range(num_length):
        for e in range(d+1, num_length):
            if num_str[d] == num_str[e]:
                for f in range (e+1, num_length):
                    if num_str[d] == num_str[f]:
                        list_.append([d, e, f])
                
    return list_

In [40]:
#This return groups of like-digits of arbitary size! It does this via recursion.
#It only returns those groups whose first like-digit is at start_index.
#The next function builds on this to generate all groups of like-digits in a number.
def find_matching_characters(string, start_index, group_size, count):
    '''Returns a list of lists; each list gives the indices of identical characters in string.
    All the lists length group_size whose first indice is start_index are return'''
    count += 1
    #return a list lists; each list has a character that matches with start_index and that character's find_matching_character
    list_ = []
    for d in range (start_index+1, len(string)):
        if string[start_index] == string[d]:
            
            if count == group_size:
                list_.append([start_index, d])
           
            else:
                for ending in find_matching_characters(string, d, group_size, count):
                    list_.append([start_index] + ending)
                    
    return list_

In [60]:
#This generates the indices of all groups of like-digits in a number.
#The lists of indices generated are fed into change_digits; together, the functions can identify 
#and simultaneously change all matching digits in a number.
def find_like_digits(num):
    '''Returns a list containing the indices of every group (of any size!) of identical digits in num.
    >>>find_like_digits(4040)
    [[0,2],[1,3]]
    >>>find_like_digits(40404)
    [[0,2],[0,4],[1,3],[0,2,4]]'''
    num_str = str(num)
    list_ = []
    for group_size in range(2, len(num_str)): #Don't let group_size = num_str, as then the number has only 1-digit so isn't prime
        for d in range(len(num_str)):
            list_ = list_ + find_matching_characters(num_str, d, group_size, 1)
    return list_
    

In [82]:
#If a new prime is formed by changing the 1st digit to 0, this doesn't count as being part of the same family of primes.
#This function checks whether the first digit of a number is zero; in my function body I will ignore such numbers.
def is_first_digit_0(num):
    '''Returns True iff the first digit of num = 0, false otherwise'''
    num_str = str(num)
    return num_str[0] == '0'

In [85]:
is_first_digit_0(099)

SyntaxError: invalid token (<ipython-input-85-c2308b6943e9>, line 1)

In [88]:
change_digit(201, 0, 0)

1

In [ ]:
if len(str(new_num)) == len(str(num)): #Check the first digit of the old prime wasn't changed to 0

In [95]:
#function body
def find_lowest_prime_in_family(family_number, max_iteration, verbose = True):
    found = False #Control termination of while loop
    prime_list = [2]
    num = 3
    iteration = 0
    while found == False and iteration < max_iteration:
        if check_if_prime(num, prime_list):
            if verbose: # Print each new-found prime
                print num
            #check if, by changing a single digit in turn, n is a family prime
            for d in range(len(str(num)) - 1): #run through digits of num except the last
                count = 0 # count the number of variations that AREN'T prime
                j = 0 # we will replace d with j, for 0<j<9
                while count <= (10 - family_number) and j < 10: #stop when we have found enough non-primes or j=10
                    new_num = change_digit(num, d, j) #replace d with j
                    if not (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if not prime
                        count += 1
                    elif len(str(new_num)) != len(str(num)): #if new number starts with a 0 (because int(07)=7)
                        count += 1 #numbers created by changing the 1st digit to 0 *don't* count as part of the family!
                    j += 1
                
                if count <= (10 - family_number): #if we don't find enough non-primes, finish
                    print 'The answer is ', num
                    
                    family = [] # record the family of primes num is in
                    for j in range(10):
                        new_num = change_digit(num, d, j) #replace d with j
                        if (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if prime
                            family.append(new_num)
                    family.sort()
                    print 'The family of primes is ', family
                    found = True #Terminate while loop

            #check if, by changing groups of like-digits simultaneously, n is a family prime
            for indice_list in find_like_digits(num): #Try every group of like-digits in turn
                count = 0 # count the number of variations that AREN'T prime
                j = 0 # we will replace d with j, for 0<j<9
                while count <= (10 - family_number) and j < 10: #stop when we have found enough non-primes or j=10
                    new_num = change_digits(num, indice_list, j) #replace like-digits with j
                    if not (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if not prime
                        count += 1
                    elif len(str(new_num)) != len(str(num)): #if new number starts with a 0 (because int(07)=7)
                        count += 1 #numbers created by changing the 1st digit to 0 *don't* count as part of the family!
                    j += 1
                
                if count <= (10 - family_number): #if we don't find enough non-primes, finish
                    print 'The answer is ', num
                    
                    family = [] # record the family of primes num is in
                    for j in range(10):
                        new_num = change_digits(num, indice_list, j) #replace like-digits with j
                        if (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if prime
                            family.append(new_num)
                    family.sort()
                    print 'The family of primes is ', family
                    print 'The list of primes lower than ', num, ' is ', prime_list
                    found = True #Terminate while loop
                    change_digits #rotate digits
        num += 2
        iteration += 1

In [97]:
#Testing
find_lowest_prime_in_family(7, 100000)

3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113
127
131
137
139
149
151
157
163
167
173
179
181
191
193
197
199
211
223
227
229
233
239
241
251
257
263
269
271
277
281
283
293
307
311
313
317
331
337
347
349
353
359
367
373
379
383
389
397
401
409
419
421
431
433
439
443
449
457
461
463
467
479
487
491
499
503
509
521
523
541
547
557
563
569
571
577
587
593
599
601
607
613
617
619
631
641
643
647
653
659
661
673
677
683
691
701
709
719
727
733
739
743
751
757
761
769
773
787
797
809
811
821
823
827
829
839
853
857
859
863
877
881
883
887
907
911
919
929
937
941
947
953
967
971
977
983
991
997
1009
1013
1019
1021
1031
1033
1039
1049
1051
1061
1063
1069
1087
1091
1093
1097
1103
1109
1117
1123
1129
1151
1153
1163
1171
1181
1187
1193
1201
1213
1217
1223
1229
1231
1237
1249
1259
1277
1279
1283
1289
1291
1297
1301
1303
1307
1319
1321
1327
1361
1367
1373
1381
1399
1409
1423
1427
1429
1433
1439
1447
1451
1453
1459
1471
1481
1483
1487
1489
1493
1499
1511

In [99]:
find_lowest_prime_in_family(8, 100000000000000)

3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113
127
131
137
139
149
151
157
163
167
173
179
181
191
193
197
199
211
223
227
229
233
239
241
251
257
263
269
271
277
281
283
293
307
311
313
317
331
337
347
349
353
359
367
373
379
383
389
397
401
409
419
421
431
433
439
443
449
457
461
463
467
479
487
491
499
503
509
521
523
541
547
557
563
569
571
577
587
593
599
601
607
613
617
619
631
641
643
647
653
659
661
673
677
683
691
701
709
719
727
733
739
743
751
757
761
769
773
787
797
809
811
821
823
827
829
839
853
857
859
863
877
881
883
887
907
911
919
929
937
941
947
953
967
971
977
983
991
997
1009
1013
1019
1021
1031
1033
1039
1049
1051
1061
1063
1069
1087
1091
1093
1097
1103
1109
1117
1123
1129
1151
1153
1163
1171
1181
1187
1193
1201
1213
1217
1223
1229
1231
1237
1249
1259
1277
1279
1283
1289
1291
1297
1301
1303
1307
1319
1321
1327
1361
1367
1373
1381
1399
1409
1423
1427
1429
1433
1439
1447
1451
1453
1459
1471
1481
1483
1487
1489
1493
1499
1511

# Solution is correct!

# Drawbacks of my solution
- when a prime p as already been found, check_if_prime(p) = False. This isn't ideal because     
-- it's counterintuitive, so makes the code harder to follow for others     
-- my code for checking whether a number is prime when rotating through primes becomes a bit clunky
- I separately replace individual digits and groups of like-digits; thus a large chunk of my code is repeated. It would be more elegant to combine these.
- I only need to replace digits that are 0, 1 or 2: I'm searching for the the *lowest* of an 8-family prime

In [101]:
import string

In [104]:
string.replace('123', '1', '5')

'523'

In [100]:
#Experimenting
max_iteration = 100000
verbose = True
family_number = 7

found = False #Control termination of while loop
prime_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
num = 101
iteration = 0
while found == False and iteration < max_iteration:
    if check_if_prime(num, prime_list):
        if verbose: # Print each new-found prime
            print num
        #check if, by changing a single digit in turn, n is a family prime
        for d in range(len(str(num)) - 1): #run through digits of num except the last
            count = 0 # count the number of variations that AREN'T prime
            j = 0 # we will replace d with j, for 0<j<9
            while count <= (10 - family_number) and j < 10: #stop when we have found enough non-primes or j=10
                new_num = change_digit(num, d, j) #replace d with j
                if not (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if not prime
                    count += 1
                j += 1

            if count <= (10 - family_number): #if we don't find enough non-primes, finish
                print 'The answer is ', num

                family = [] # record the family of primes num is in
                for j in range(10):
                    new_num = change_digit(num, d, j) #replace d with j
                    if (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if prime
                        family.append(new_num)
                family.sort()
                print 'The family of primes is ', family
                found = True #Terminate while loop

        #check if, by changing groups of like-digits simultaneously, n is a family prime
        for indice_list in find_like_digits(num): #Try every group of like-digits in turn
            count = 0 # count the number of variations that AREN'T prime
            j = 0 # we will replace d with j, for 0<j<9
            while count <= (10 - family_number) and j < 10: #stop when we have found enough non-primes or j=10
                new_num = change_digits(num, indice_list, j) #replace like-digits with j
                if not (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if not prime
                    count += 1
                j += 1

            if count <= (10 - family_number): #if we don't find enough non-primes, finish
                print 'The answer is ', num

                family = [] # record the family of primes num is in
                for j in range(10):
                    new_num = change_digits(num, indice_list, j) #replace like-digits with j
                    if (check_if_prime(new_num,prime_list, add=False) or new_num in prime_list): #if prime
                        family.append(new_num)
                family.sort()
                print 'The family of primes is ', family
                found = True #Terminate while loop
                change_digits #rotate digits
    num += 2
    iteration += 1

101
103
107
109
113
127
131
137
139
149
151
157
163
167
173
179
181
191
193
197
199
211
223
227
229
233
239
241
251
257
263
269
271
277
281
283
293
307
311
313
317
331
337
347
349
353
359
367
373
379
383
389
397
401
409
419
421
431
433
439
443
449
457
461
463
467
479
487
491
499
503
509
521
523
541
547
557
563
569
571
577
587
593
599
601
607
613
617
619
631
641
643
647
653
659
661
673
677
683
691
701
709
719
727
733
739
743
751
757
761
769
773
787
797
809
811
821
823
827
829
839
853
857
859
863
877
881
883
887
907
911
919
929
937
941
947
953
967
971
977
983
991
997
1009
1013
1019
1021
1031
1033
1039
1049
1051
1061
1063
1069
1087
1091
1093
1097
1103
1109
1117
1123
1129
1151
1153
1163
1171
1181
1187
1193
1201
1213
1217
1223
1229
1231
1237
1249
1259
1277
1279
1283
1289
1291
1297
1301
1303
1307
1319
1321
1327
1361
1367
1373
1381
1399
1409
1423
1427
1429
1433
1439
1447
1451
1453
1459
1471
1481
1483
1487
1489
1493
1499
1511
1523
1531
1543
1549
1553
1559
1567
1571
1579
1583
1597
1601
1607
160